# Introduction

<center><h3>**Welcome to the Summarization Notebook.**</h3></center>

In this assignment, you are going to train a neural network to summarize news articles.
Your neural network is going to learn from example, as we provide you with (article, summary) pairs.
We provide you with a **toy dataset** made of only articles about police related news.
Usual datasets can be 20x larger in size, but we have reduced it for computational purposes.

You will do this using a Transformer network, from the __[Attention is all you need](http://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf)__ paper.
In this assignment you will:
- Learn to process text into sub-word tokens, to avoid fixed vocabulary sizes, and UNK tokens.
- Implement the key conceptual blocks of a Transformer.
- Use a Transformer to read a news article, and produce a summary.
- Perform operations on learned word-vectors to examine what the model has learned.

    
** Before you start **

You should read the Attention is all you need paper.
We are providing you with skeleton code for the Transformer, but there will have to implement 5 conceptual blocks of the transformer yourself:
-  AttentionQKV: the Query, Key, Value attention mechanism at the center of the Transformer
- MultiHeadAttention: the multiple heads that enable each input to attend at many places at once.
- PositionEmbedding: the sinusoid-based position embedding of the Transformer.
- Encoder & Decoder: The encoder (that reads inputs, such as news articles), the decoder (that produces the output summary, one token at a time)
- Full Transformer: piecing it all together.

# Library imports

In [1]:
# This cell autoreloads the notebook when you change you python file code.
# If you think the notebook did not reload, rerun this cell.
%load_ext autoreload
%autoreload 2

In [2]:
root_folder = ""
dataset_folder = "dataset/"

In [5]:
import os
import sys
sys.path.append(root_folder)
from transformer import Transformer
import sentencepiece as spm
import torch as th
from torch import nn
from torch.nn import functional as F
from torch import optim
import numpy as np
import json
import capita
import os
from transformer_utils import set_device
import gc
from utils import validate_to_array, model_out_to_list
from tqdm import tqdm

device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)
set_device(device)

list_to_device = lambda th_obj: [tensor.to(device) for tensor in th_obj]

cuda


In [6]:
# Load the word piece model that will be used to tokenize the texts into
# word pieces with a vocabulary size of 10000
sp = spm.SentencePieceProcessor()
sp.Load(root_folder+"dataset/wp_vocab10000.model")

vocab = [line.split('\t')[0] for line in open(root_folder+"dataset/wp_vocab10000.vocab", "r")]
pad_index = vocab.index('#')

def pad_sequence(numerized, pad_index, to_length):
    pad = numerized[:to_length]
    padded = pad + [pad_index] * (to_length - len(pad))
    mask = [w != pad_index for w in padded]
    return padded, mask

# Building blocks of a Transformer


**TODO**:

Implement the 5 blocks of the Transformer. In order to finish this section, you should get very small error <1e-7 on each of the 5 checks in this section.


The Transformer is split into 3 files: transformer_attention.py, transformer_utils.py and transformer.py

Each section below gives you directions and a way to verify your code works properly.

You do not need to modify the rest of the code provided, but should read it to understand overall architecture.

Our Transformer is built as a Pytorch model, a standard that is good for you to get accustomed to.



## (1) Implementing the Query-Key-Value Attention (AttentionQKV)

This part is located in `AttentionQKV` in `transformer_attention.py`. You must implement the call function of the class.
You will need to implement the mathematical procedure of AttentionQKV that is described in the [Attention is all you need paper](https://arxiv.org/pdf/1706.03762.pdf).

In [20]:
from transformer_attention import AttentionQKV

batch_size = 2
n_queries = 3
n_keyval = 5
depth_k = 2
depth_v = 2

with open(root_folder+"transformer_checks/attention_qkv_io.json", "r") as f:
    io = json.load(f)
    queries = th.tensor(io['queries'])
    keys = th.tensor(io['keys'])
    values = th.tensor(io['values'])
    expected_output  = th.tensor(io['output'])
    expected_weights = th.tensor(io['weights'])

attn_qkv = AttentionQKV()
output, weights = attn_qkv(queries, keys, values)
validate_to_array(model_out_to_list,((queries,keys,values),attn_qkv),'attentionqkv', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-output)).item(), "(should be 0.0 or close to 0.0)")
print("Total error on the weights:",th.sum(th.abs(expected_weights-weights)).item(), "(should be 0.0 or close to 0.0)")

Total error on the output: 3.0547380447387695e-07 (should be 0.0 or close to 0.0)
Total error on the weights: 3.0174851417541504e-07 (should be 0.0 or close to 0.0)


## (2) Implementing Multi-head attention

This part is located in the class `MultiHeadProjection` in `transformer_attention.py`.
You must implement the call, \_split_heads, and \_combine_heads functions.

**Procedure**

The objective is to leverage the `AttentionQKV` class you already wrote.

Your input are the queries, keys, values as 3-d tensors (batch_size, sequence_length, feature_size).

Split them into 4-d tensors (batch_size, n_heads, sequence_length, new_feature_size). Where:
$$feature\_size = n\_heads * new_{feature\_size}.$$

You can then feed the split qkv to your implemented `AttentionQKV`, which will treat each head as an independent attention function.

Then the output must be combined back into a 3-d tensor.
You can test the validity of your implementation in the cell below.

In [24]:
from transformer_attention import MultiHeadProjection

batch_size = 2
n_queries = 3
n_heads = 4
n_keyval = 5
depth_k = 8
depth_v = 8

with open(root_folder+"transformer_checks/multihead_io.json", "r") as f:
    io = json.load(f)
    queries = th.tensor(io['queries'])
    keys = th.tensor(io['keys'])
    values = th.tensor(io['values'])
    expected_output  = th.tensor(io['output'])

mhp = MultiHeadProjection(n_heads, (depth_k,depth_v))
multihead_output = mhp((queries, keys, values))
validate_to_array(model_out_to_list,(((queries,keys,values),),mhp),'multihead', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-multihead_output)).item(), "(should be 0.0 or close to 0.0)")

Total error on the output: 1.5338882803916931e-06 (should be 0.0 or close to 0.0)


## (3) Position Embedding 

You must implement the `FeedForward` and `PositionEmbedding` classes in `transformer.py`.


The cell below helps you verify the validity of your implementation


In [35]:
from transformer import PositionEmbedding

batch_size = 2
sequence_length = 3
dim = 4

with open(root_folder+"transformer_checks/position_embedding_io.json", "r") as f:
    io = json.load(f)
    inputs = th.tensor(io['inputs'])
    expected_output  = th.tensor(io['output'])

pos_emb = PositionEmbedding(dim)
(inputs,expected_output,pos_emb) = list_to_device((inputs,expected_output,pos_emb))
output_t = pos_emb(inputs)
validate_to_array(model_out_to_list,((inputs,),pos_emb),'position_embedding', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-output_t)).item(), "(should be 0.0 or close to 0.0)")

Total error on the output: 2.980232238769531e-07 (should be 0.0 or close to 0.0)


## (4) Transformer Encoder / Transformer Decoder

You now have all the blocks needed to implement the Transformer.
For this part, you have to fill in 2 classes in the `transformer.py` file: `TransformerEncoderBlock`, `TransformerDecoderBlock`.

The code below will verify the accuracy of each block

In [36]:
from transformer import TransformerEncoderBlock

batch_size = 2
sequence_length = 5
hidden_size = 6
filter_size = 12
n_heads = 2

with open(root_folder+"transformer_checks/transformer_encoder_block_io_new.json", "r") as f:
    io = json.load(f)
    inputs = th.tensor(io['inputs'])
    expected_output = th.tensor(io['output'])
enc_block = TransformerEncoderBlock(input_size=6, n_heads=n_heads, filter_size=filter_size, hidden_size=hidden_size)
# th.save(enc_block.state_dict(),root_folder+"transformer_checks/transformer_encoder_block")
enc_block.load_state_dict(th.load(root_folder+"transformer_checks/transformer_encoder_block"))
(inputs,expected_output,enc_block) = list_to_device((inputs,expected_output,enc_block))
output_t = enc_block(inputs)
validate_to_array(model_out_to_list,((inputs,),enc_block),'encoder_block', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-output_t)).item(), "(should be 0.0 or close to 0.0)")

Total error on the output: 0.013845548033714294 (should be 0.0 or close to 0.0)


In [37]:
from transformer import TransformerDecoderBlock
batch_size = 2
encoder_length = 5
decoder_length = 3
hidden_size = 6
filter_size = 12
n_heads = 2

with open(root_folder+"transformer_checks/transformer_decoder_block_io_new.json", "r") as f:
    io = json.load(f)
    decoder_inputs = th.tensor(io['decoder_inputs'])
    encoder_output = th.tensor(io['encoder_output'])
    expected_output = th.tensor(io['output'])

dec_block = TransformerDecoderBlock(input_size=6, n_heads=n_heads, filter_size=filter_size, hidden_size=hidden_size)
dec_block.load_state_dict(th.load(root_folder+"transformer_checks/transformer_decoder_block"))
(decoder_inputs,encoder_output,expected_output,dec_block) = list_to_device((decoder_inputs,encoder_output,expected_output,dec_block))
output_t = dec_block(decoder_inputs, encoder_output)
validate_to_array(model_out_to_list,((decoder_inputs, encoder_output),dec_block),'decoder_block', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-output_t)).item(), "(should be 0.0 or close to 0.0)")


Total error on the output: 0.009545251727104187 (should be 0.0 or close to 0.0)


## (5) Transformer

This is the final high-level function that pieces it all together.

You have to implement the call function of the `Transformer` class in the `transformer.py` file.

The block below verifies your implementation is correct.

In [38]:
from transformer import Transformer

batch_size = 2
vocab_size = 11
n_layers = 3
n_heads = 4
d_model = 8
d_filter = 16
input_length = 5
output_length = 3

with open(root_folder+"transformer_checks/transformer_io_new.json", "r") as f:
    io = json.load(f)
    enc_input = th.tensor(io['enc_input'])
    dec_input = th.tensor(io['dec_input'])
    enc_mask = th.tensor(io['enc_mask'])
    dec_mask = th.tensor(io['dec_mask'])
    expected_output = th.tensor(io['output'])
transformer = Transformer(vocab_size=vocab_size, n_layers=n_layers, n_heads=n_heads, d_model=d_model, d_filter=d_filter)
transformer.load_state_dict(th.load(root_folder+"transformer_checks/transformer"))
(enc_input,dec_input,enc_mask,dec_mask,expected_output,transformer) \
    = list_to_device((enc_input,dec_input,enc_mask,dec_mask,expected_output,transformer))
output_t = transformer(enc_input, target_sequence=dec_input, encoder_mask=enc_mask, decoder_mask=dec_mask)
validate_to_array(model_out_to_list, ((enc_input, dec_input, enc_mask, dec_mask),transformer),'transformer', root_folder)
print("Total error on the output:",th.sum(th.abs(expected_output-output_t)).item(), "(should be 0.0 or close to 0.0)")

Total error on the output: 0.12164890766143799 (should be 0.0 or close to 0.0)


# Training the model

Your objective is to train the Language on the dataset you are provided to reach a **validation loss <= 6.50**

Careful: we will be testing this loss on an unreleased test set, so make sure to evaluate properly on a validation set and not overfit.

You must save the model you want us to test under: models/final_transformer_summarization (the .index, .meta and .data files)

**Advice**:
- It should be possible to attain validation loss <= 6.50 with the model dimensions we've specified (n_layers=6, d_model=104, d_filter=416), but you can tune these hyperparameters. Increasing d_model will yield better model, at the cost of longer training time.
- You should try tuning the learning rate, as well as what optimizer you use.
- You might need to train for a few (up to 2 hours) to obtain our expected loss. Remember to tune your hyperparameters first, once you find ones that work well, let it train for longer.


In [39]:
with open(root_folder+"dataset/summarization_dataset_preprocessed.json", "r") as f:
    dataset = json.load(f)

# We load the dataset, and split it into 2 sub-datasets based on if they are training or validation.
# Feel free to split this dataset another way, but remember, a validation set is important, to have an idea of 
# the amount of overfitting that has occurred!

d_train = [d for d in dataset if d['cut'] == 'training']
d_valid = [d for d in dataset if d['cut'] == 'evaluation']

len(d_train), len(d_valid)

(61055, 1558)

In [40]:
# An example (article, summary) pair in the training data:

print(d_train[145]['story'])
print("=======================\n=======================")
print(d_train[145]['summary'])

Tbilisi, Georgia (CNN)Police have shot and killed a white tiger that killed a man Wednesday in Tbilisi, Georgia, a Ministry of Internal Affairs representative said, after severe flooding allowed hundreds of wild animals to escape the city zoo. 
The tiger attack happened at a warehouse in the city center. The animal had been unaccounted for since the weekend floods destroyed the zoo premises.
The man killed, who was 43, worked in a company based in the warehouse, the Ministry of Internal Affairs said. Doctors said he was attacked in the throat and died before reaching the hospital. 
Experts are still searching the warehouse, the ministry said, adding that earlier reports that the tiger had injured a second man were unfounded. 
The zoo administration said Wednesday that another tiger was still missing. It was unable to confirm if the creature was dead or had escaped alive.
Georgian Prime Minister Irakli Garibashvili apologized to the public, saying he had been misinformed by the zoo's ma

Similarly to the previous assignment, we create a function to get a random batch to train on, given a dataset.

In [41]:
def build_batch(dataset, batch_size):
    indices = list(np.random.randint(0, len(dataset), size=batch_size))
    
    batch = [dataset[i] for i in indices]
    batch_input = np.array([a['input'] for a in batch])
    batch_input_mask = np.array([a['input_mask'] for a in batch])
    batch_output = np.array([a['output'] for a in batch])
    batch_output_mask = np.array([a['output_mask'] for a in batch])
    
    return batch_input, batch_input_mask, batch_output, batch_output_mask

We now instantiate the Transformer with our sets of hyperparameters specific to the task of summarization.
In summarization, we are going to go from documents with up to 400 words, to documents with up to 100 words.
The vocabulary size is set for you, and is of 10,000 words (we are using WordPieces, [here is a paper about subword encoding](http://aclweb.org/anthology/P18-1007), if you are interested).

In [45]:
# fill in the trainer with similar logic to the first notebook's training loop

class TransformerTrainer(nn.Module):
    def __init__(self, vocab_size, d_model, input_length, output_length, n_layers, d_filter, dropout=0, learning_rate=1e-3):
        super().__init__()
        self.model = Transformer(vocab_size=vocab_size, d_model=d_model, n_layers=n_layers, d_filter=d_filter)

        # Summarization loss
        criterion = nn.CrossEntropyLoss(reduce='none')
        self.loss_fn = lambda pred,target,mask: (criterion(pred.permute(0,2,1),target)*mask).sum()/mask.sum()
        self.learning_rate = learning_rate
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
    def forward(self,batch,optimize=True):
        pred_logits = self.model(**batch)
        target,mask = batch['target_sequence'],batch['decoder_mask']
        loss = self.loss_fn(pred_logits,target.long(),mask)
        accuracy = (th.eq(pred_logits.argmax(dim=2,keepdim=False),target).float()*mask).sum()/mask.sum()
        
        if optimize:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
                
        return loss, accuracy

In [46]:
# Dataset related parameters
vocab_size = len(vocab)
ilength = 400 # Length of the article
olength  = 100 # Length of the summaries

# Model related parameters, feel free to modify these.
n_layers = 6
d_model  = 160
d_filter = 4*d_model
batch_size = 16

dropout = 0
learning_rate = 1e-3
trainer = TransformerTrainer(vocab_size, d_model, ilength, olength, n_layers, d_filter, dropout)
model_id = 'test1'
os.makedirs(root_folder+'models/part2/',exist_ok=True)

c:\ProgramData\anaconda3\envs\cs182_hw3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [47]:
# Skeleton code, as in the previous notebook.
# Write code training code and save your best performing model on the
# validation set. We will be testing the loss on a held-out test dataset.
from tqdm import tqdm

trainer.to(device)
trainer.model.train()
losses,accuracies = [],[]
t = tqdm(range(int(1e3)+1))
for i in t:
    # Create a random mini-batch from the training dataset
    batch = build_batch(d_train, batch_size)
    # Build the feed-dict connecting placeholders and mini-batch
    batch_input, batch_input_mask, batch_output, batch_output_mask = [th.tensor(tensor) for tensor in batch]
    batch_input, batch_input_mask, batch_output, batch_output_mask \
                = list_to_device([batch_input, batch_input_mask, batch_output, batch_output_mask])
    batch = {'source_sequence': batch_input, 'target_sequence': batch_output,
            'encoder_mask': batch_input_mask, 'decoder_mask': batch_output_mask}

    # Obtain the loss. Be careful when you use the train_op and not, as previously.
    train_loss, accuracy = trainer(batch)
    losses.append(train_loss.item()),accuracies.append(accuracy.item())
    if i % 10 == 0:
        t.set_description(f"Iteration: {i} Loss: {np.mean(losses[-10:])} Accuracy: {np.mean(accuracies[-10:])}")
    if i % 100 == 0:
        save_dict = dict(
            kwargs = dict(
                vocab_size=vocab_size,
                d_model=d_model,
                n_layers=n_layers, 
                d_filter=d_filter
            ),
            model_state_dict = trainer.model.state_dict(),
            notes = ""
        )
        th.save(save_dict, root_folder+f'models/part2/model_{model_id}.pt')

Iteration: 1000 Loss: 4.0584046125411986 Accuracy: 0.2785654544830322: 100%|██████████| 1001/1001 [02:36<00:00,  6.39it/s]


# Using the Summarization model

Now that you have trained a Transformer to perform Summarization, we will use the model on news articles from the wild.

The three subsections below explore what the model has learned.

## The validation loss

Measure the validation loss of your model. This part could be used, as in our previous notebook, in deciding what is a likely, vs. unlikely summary for an article.

We will use the code here with the unreleased test-set to evaluate your model.

In [48]:
gc.collect()
model_id = "test1"
save_dict = th.load(root_folder+'models/part2/'+f"model_{model_id}.pt", map_location='cpu')
model = Transformer(**save_dict['kwargs'])
model.load_state_dict(save_dict['model_state_dict'])
trainer.model = model.eval()
trainer.to(device)

TransformerTrainer(
  (model): Transformer(
    (encoder): TransformerEncoder(
      (embedding_layer): TransformerInputEmbedding(
        (embedding): Embedding(10000, 160)
        (position_encoding): PositionEmbedding()
        (dropout): Dropout(p=0, inplace=False)
      )
      (encoder0): TransformerEncoderBlock(
        (norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttention(
          (attention_layer): MultiHeadProjection(
            (attention_map): AttentionQKV(
              (apply_mask): ApplyAttentionMask()
            )
          )
          (query_layer): Linear(in_features=160, out_features=160, bias=False)
          (key_layer): Linear(in_features=160, out_features=160, bias=False)
          (value_layer): Linear(in_features=160, out_features=160, bias=False)
          (output_layer): Linear(in_features=160, out_features=160, bias=False)
        )
        (feed_forward): TransformerFeedForward(
          (norm): Laye

In [49]:
losses = []
for i in tqdm(range(100)):
    batch = build_batch(d_valid, 1)
    # Build the feed-dict connecting placeholders and mini-batch
    batch_input, batch_input_mask, batch_output, batch_output_mask = [th.tensor(tensor) for tensor in batch]
    batch_input, batch_input_mask, batch_output, batch_output_mask \
                = list_to_device([batch_input, batch_input_mask, batch_output, batch_output_mask])
    batch = {'source_sequence': batch_input, 'target_sequence': batch_output,
            'encoder_mask': batch_input_mask, 'decoder_mask': batch_output_mask}
    valid_loss, accuracy = trainer(batch,optimize=False)
    losses.append(float(valid_loss.cpu().item()))
print("Validation loss:", np.mean(losses))

100%|██████████| 100/100 [00:04<00:00, 24.17it/s]

Validation loss: 5.0475062716007235


In [50]:
# Your best performing model should go here.
os.makedirs(root_folder+"best_models",exist_ok=True)
best_model_file = root_folder+"best_models/part2_best_model.pt"
th.save(save_dict,best_model_file)

## Generating an article's summary

This model we have built is meant to be used to generate summaries for new articles we do not have summaries for.
We got a [news article](https://www.chicagotribune.com/news/local/breaking/ct-met-officer-shot-20190309-story.html) from the Chicago Tribune about a police shooting, and want to use our model to produce a summary.

As you will see, our model is still limited in its ability, and will most likely not produce an interpretible summary, however, with more data and training, this model would be able to produce good summaries.

In [51]:
article_text = "A 34-year-old Chicago police officer has been shot in the shoulder during the execution of a search warrant in the Humboldt Park neighborhood, police say. The alleged shooter, a 19-year-old woman, was in custody. The shooting happened about 7:20 p.m. in the 2700 block of West Potomac Avenue, police said. The officer, part of the Grand Central District tactical unit, was taken to Stroger Hospital. While officers were serving a \"typical\" search warrant for \"narcotics and illegal weapons\" and were attempting to reach a rear door, \"a shot was fired,\" striking the tactical officer in the shoulder, said Chicago police Superintendent Eddie Johnson during a news briefing outside the hospital. He said the officer, who has about four or five years on the job, was \"stable\" but in critical condition. \"His family is here,\" Johnson said. \"He’s talking a lot and just wants the ordeal to be over.\" He said this incident serves as just another reminder of how dangerous a police officer’s job is. At the scene of the shooting, crime tape closed Potomac from Washtenaw Avenue to California Avenue and encompassed the alley west of the brick apartment building, south of Potomac. Dozens of officers stood in the alley, while even more walked up and down the street. Neighbors gathered at the edge of the yellow tape on the sidewalk along California and watched them work. Standing next to a man, a woman talked to police in the crime scene, across the street. \"We're not under arrest? We can go?\" the woman checked with officers. They told her she could go, and she and the man walked underneath the yellow tape and out of the crime scene."
input_length = 400
output_length = 100

# Process the capitalization with the preprocess_capitalization of the capita package.
article_text = capita.preprocess_capitalization(article_text)

# Numerize the tokens of the processed text using the loaded sentencepiece model.
numerized = sp.EncodeAsIds(article_text)
# Pad the sequence and keep the mask of the input
padded, mask = pad_sequence(numerized, pad_index, input_length)

# Making the news article into a batch of size one, to be fed to the neural network.
encoder_input = np.array([padded])
encoder_mask = np.array([mask])

decoded_so_far = [0]

for j in range(output_length):
    padded_decoder_input, decoder_mask = pad_sequence(decoded_so_far, pad_index, output_length)
    padded_decoder_input = [padded_decoder_input]
    decoder_mask = [decoder_mask]
    print("========================")
    print(padded_decoder_input)
    # Use the model to find the distrbution over the vocabulary for the next word
    batch = (encoder_input,encoder_mask,padded_decoder_input,decoder_mask)
    batch_input, batch_input_mask, batch_output, batch_output_mask = [th.tensor(tensor) for tensor in batch]
    batch_input, batch_input_mask, batch_output, batch_output_mask \
                = list_to_device([batch_input, batch_input_mask, batch_output, batch_output_mask])
    batch = {'source_sequence': batch_input, 'target_sequence': batch_output,
            'encoder_mask': batch_input_mask, 'decoder_mask': batch_output_mask}
    logits = trainer.model(**batch).cpu().detach().numpy()

    chosen_words = np.argmax(logits, axis=2) # Take the argmax, getting the most likely next word
    decoded_so_far.append(int(chosen_words[0, j])) # We add it to the summary so far


print("The final summary:")
print("".join([vocab[i] for i in decoded_so_far]).replace("▁", " "))

[[0, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998]]
[[0, 3, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 9998, 99

## Word vectors

The model we train learns word representations for each word in our vocabulary. A word represention is a vector of **dim** size.

It is common in NLP to inspect the word vectors, as some properties of language often appear in the embedding structure.


We are going to load the word embeddings learned by our model, and inspect it.
Because our network was not trained for long, we are going for the simplest patterns, but if we let the network train longer, it learns more complex, semantic patterns.

In [52]:
# We help you load the matrix, as it is hidden within the Transformer structure.
E = trainer.model.encoder.embedding_layer.embedding.weight.cpu().detach().numpy()

print("The embedding matrix has shape:", E.shape)
print("The vocabulary has length:", len(vocab))

The embedding matrix has shape: (10000, 160)
The vocabulary has length: 10000


Pronouns serve very similar purposes, therefore we should expect the representation of "he" and "she" to be similar, and have cosine similarity.

- **TODO**:  Find the cosine similarity between the vectors that represent words "she" and "he".
- **TODO**:  Find the cosine similarity between the vectors that represent words "more" and "less".

We can contrast that with the cosine similarity to a random, non-related word, like "ball", or "gorilla".
- **TODO**: Compute the cosine similarity between "she" and "ball".
- **TODO**: Compute the cosine similarity between "more" and "protest".



In [53]:
def cosine_sim(v1, v2):
    # TODO: Implement the cosine similarity of 2 vectors. Careful: the words might not have unit norm.
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return v1 * v2 / (norm_v1 * norm_v2)

for w1, w2 in [("she", "he"), ("more", "less"), ("she", "ball"), ("more", "gorilla")]:
    w1_index = vocab.index('▁'+w1) # The index of the first  word in our vocabulary
    w2_index = vocab.index('▁'+w2) # The index of the second word in our vocabulary
    w1_vec = E[w1_index] # Get the embedding vector of the first  word
    w2_vec = E[w2_index] # Get the embedding vector of the second word
    
    print(w1," vs. ", w2, "similarity:",cosine_sim(w1_vec, w2_vec))
validate_to_array(lambda f,i: (f(*i),i), (cosine_sim, tuple(20*np.random.random((2,1000))-1)),'cosine_sim', root_folder) 

she  vs.  he similarity: [ 5.74290846e-03 -3.51148978e-04  3.38591984e-03  3.33328471e-02
 -1.40976282e-02  9.95143130e-03 -6.08776929e-03  5.61436638e-03
 -4.38284501e-03 -1.06377876e-03  6.41978858e-03 -7.86374882e-03
 -5.73210092e-03  3.10137961e-03  4.37364588e-03 -2.16028141e-03
  4.48146556e-03  2.86388764e-04  2.79442105e-03 -4.28341283e-03
  5.95713027e-05  7.44383957e-04  2.34460924e-03  2.95588607e-03
  2.12525460e-03  1.16153993e-03 -1.12121301e-02 -8.39031942e-04
 -1.17611815e-03  5.56286052e-03 -6.21923339e-03 -1.65181584e-03
 -4.00981342e-04  7.40840938e-03  1.23257656e-02  1.53429981e-03
 -6.08333293e-03 -1.95745705e-03 -3.14890710e-03 -2.09334609e-03
  1.32090307e-03  8.42344761e-03 -4.23761504e-03 -3.08754807e-03
 -8.46188795e-03  1.13363250e-03  2.35767802e-03 -2.13482603e-03
  1.89591525e-03 -7.00600259e-03  3.48593527e-03  6.48747478e-03
  7.78688770e-03 -4.02248872e-04  1.28466019e-03 -7.04486249e-03
  4.80706478e-03  5.29767992e-03 -5.92028773e-05  9.49201919e-03


c:\ProgramData\anaconda3\envs\cs182_hw3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


These effects are unfortunately small, as we have only trained the network on a few hours on a few thousand articles.
However, the same model trained for longer on more data exhibits many interesting semantic and syntactic patterns, such as:

- Words vectors with high cosine similarity usually represent words that have semantic similarity (such as duck and pigeon)
- Analogies can occur, a famous case is that of: woman - man + king ≈ queen. Or france - paris + rome ≈ italy.

- Looking at top-k similar words can help find synonyms.

To read examples of more complex patterns that appear in word embedding spaces, read [this blog](https://explosion.ai/blog/sense2vec-with-spacy). To play with a live demo and try similarities on rich word embeddings, [go here.](https://explosion.ai/demos/sense2vec)